<img src="../../shared/img/slides_banner.svg" width=2560></img>

# Multi-way Modeling 02

In [ ]:
import sys

sys.path.append("../../")

from shared.src import quiet
from shared.src import seed
from shared.src import style

In [ ]:
from pathlib import Path
import random

from IPython.display import HTML, Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import seaborn as sns
import scipy.stats

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats

In [ ]:
sns.set_context("notebook", font_scale=1.7)

In [ ]:
import shared.src.utils.util as shared_util

In [ ]:
def make_plot(mus, ax=None, **plot_kwargs):
    if ax is None:
        f, ax = plt.subplots(figsize=(12, 6))
    xs = np.arange(mus.shape[0])
    ax.plot(xs, mus[:, 0], lw=4, color="C0", **plot_kwargs)
    ax.plot(xs, mus[:, 1], lw=4, color="C1", **plot_kwargs)

    
from matplotlib.lines import Line2D


def make_line(color, linewidth=4):
    return Line2D([0], [0], linewidth=linewidth, color=color)

# Previously, we worked with data that varied only along one category, or factor.

For example:
- how does a participant's performance on a task vary depending on whether they are given caffeine or not?
- how do a participant's brain activity patterns vary with the type of music they are listening to?

## In these lectures, we will learn how to work with data that varies along two or more categories, which might _interact_.

For example:
- how do whether a participant drinks caffeine and whether they're over 40 interact to determine their performance on a task?
- if we show a person movies inside an fMRI machine, how do both the sound and the image being presented together determine the pattern of activity in their brain?

# Let's look for interactions in some real data.

## First, a toy dataset:

I ran a version of the "accuracy" experiment described in the other set of slides:
with my left and right eyes open and closed,
I threw

In [ ]:
toss_data = pd.DataFrame({"success":
                          [1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
                           0, 1, 1, 0, 1, 0, 1, 1, 1, 1,
                           1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
                           1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              "left_eye_closed": [0] * 10 + [0] * 10 + [1] * 10 + [1] * 10,
              "right_eye_closed": [0] * 10 + [1] * 10 + [0] * 10 + [1] * 10})

In [ ]:
print(toss_data.groupby(["left_eye_closed", "right_eye_closed"]).mean())

In [ ]:
f, ax = plt.subplots(figsize=(12, 6))
sns.pointplot(x="left_eye_closed", y="success", hue="right_eye_closed",
              data=toss_data, scale=2, errwidth=6);

In [ ]:
with pm.Model() as accuracy_model:
    accuracies = pm.Uniform("accuracies", shape=(2, 2))
    
    successes = pm.Bernoulli(
        "successes",
        p=accuracies[toss_data["right_eye_closed"], toss_data["left_eye_closed"]],
        observed=toss_data["success"])

In [ ]:
with accuracy_model:
    accuracy_trace = pm.sample()

In [ ]:
accuracy_df = shared_util.samples_to_dataframe(accuracy_trace)

In [ ]:
pm.plot_posterior(accuracy_trace, figsize=(12, 12), text_size=16, ref_val=0);

In [ ]:
def compute_interaction_effect(accuracies):
    baseline = accuracies[0, 0]
    delta_left_eye = accuracies[0, 1] - baseline
    delta_right_eye = accuracies[1, 0] - baseline
    
    prediction_from_separate = baseline + delta_left_eye + delta_right_eye
    actually_observed_value = accuracies[1, 1]
    
    return actually_observed_value - prediction_from_separate

In [ ]:
interaction_effect_posterior = accuracy_df["accuracies"].apply(compute_interaction_effect)

In [ ]:
f, ax = plt.subplots(figsize=(12, 6))
sns.distplot(interaction_effect_posterior, label="posterior",
             axlabel="Interaction Effect of Closing Left and Right Eye");
ax.hlines(0, *pm.stats.hpd(interaction_effect_posterior), lw=6)
ax.legend();

In [ ]:
(interaction_effect_posterior > 0).mean()

### Important: we don't need to just think about differences in means.

Any difference in parameters across multiple groups,
e.g. the difference in `p` above,
can be examined in a multi-way model,
not just differences in means.

For example,
we could look at differences in standard deviations,
or in the `p` parameter of a `ZeroInflatedPoisson`,
across multiple categories.

## Now, a dataset from a real psychological experiment:

We'll be using some EEG experiment data graciously provided by the [Voytek lab](http://voyteklab.com/about-us/) of UCSD. Participants of varying ages were asked to perform a working memory task with varying levels of difficulty. The raw EEG signal has been summarized into the following two measures:

* [Contralateral Delay Activity](https://www.ncbi.nlm.nih.gov/pubmed/26802451), or CDA, is used to measure the engagement of visual working memory.

* [Frontal Midline Theta](https://www.ncbi.nlm.nih.gov/pubmed/9895201) oscillation amplitude has been correlated with sustained, internally-directed cognitive activity.

The performance of the subjects has also been summarized using the measure
[d'](https://en.wikipedia.org/wiki/Sensitivity_index) (pronounced "d-prime"),
also known as the *sensitivity index*.
d' is a measure of the subject's performance in a task,
based on the "linear signal model" we've looked at previously.

In this lecture, we will look at `d`, the subject performance metric.

In [ ]:
shared_data_path = Path("..") / ".." / "shared" / 'data'

df = pd.read_csv(shared_data_path / 'voytek_working_memory_aging_split.csv', index_col=None)

print(df.sample(5))

In [ ]:
print(df.groupby("group")["age"].describe())

In [ ]:
f, ax = plt.subplots(figsize=(12, 6))
sns.distplot(df["d"], ax=ax);

As we've been doing,
we'd like to try breaking this data down
by thinking of it as a mixture distribution,
using the categories as the mixture elements.

# If we split this data up by one variable at a time, we know what to do.

We have a frequentist approach, based on one-way $F$ tests:

In [ ]:
f, ax = plt.subplots(figsize=(12, 6))
sns.violinplot(y="d", x="group", data=df, ax=ax, linewidth=4, width=0.3);

In [ ]:
df.groupby(["group"])["d"].mean()

In [ ]:
scipy.stats.f_oneway(df["d"][df["group"] == 1],
                     df["d"][df["group"] == 2])

This approach works with more than two groups,
unlike the first approach we considered, the $t$-test.

In [ ]:
f, ax, = plt.subplots(figsize=(12, 6))
sns.violinplot(y="d", x="difficulty", data=df, ax=ax, linewidth=4, width=0.3);

In [ ]:
df.groupby(["difficulty"])["d"].mean()

In [ ]:
scipy.stats.f_oneway(df["d"][df["difficulty"] == 1],
                     df["d"][df["difficulty"] == 2],
                     df["d"][df["difficulty"] == 3])

# We've already worked with one-way models in pyMC.

First, let's simplify and format our data.

In [ ]:
data = pd.DataFrame()

data["age_group"] = df["group"] - 1  # subtract 1 so that it starts from 0, like Python indexing
data["difficulty"] = df["difficulty"] - 1
# watch out: we can't use boolean Series as our indexers!

data["d"] = df["d"]

## For a one-way model, we define something like a list of parameters, then index into that list.

$$
\mu \sim \text{Normal}(0, 1\mathrm{e}6, \text{shape}=3)\\
\sigma \sim \text{Exponential}(0.1)\\
d \sim \text{Normal}(\mu[i], \sigma)
$$

where $i$ is a variable that indexes into the "list" $\mu$,
selecting out the appropriate mean for each datapoint.

In [ ]:
difficulty_indexer = data["difficulty"]

In [ ]:
with pm.Model() as eeg_difficulty_model:
    means = pm.Normal("mus", mu=0, sd=1e6, shape=3)
    sd = pm.Exponential("sigma", lam=0.1)
    
    observations = pm.Normal("d", mu=means[difficulty_indexer], sd=sd, observed=data["d"])

In class, we won't sample from and work with these models,
since we've already seen them,
but feel free to add cells and look at the posteriors in your copy of the slides.

### For a different one-way model, we define a different indexer and change the shapes.

$$
\mu[j] \sim \text{Normal}(0, 1\mathrm{e}6, \text{shape}=2)\\
\sigma \sim \text{Exponential}(0.1)\\
d \sim \text{Normal}(\mu[j], \sigma)
$$

In [ ]:
age_indexer = data["age_group"]

In [ ]:
with pm.Model() as eeg_age_model:
    means = pm.Normal("mus", mu=0, sd=1e6, shape=2)
    sd = pm.Exponential("sigma", lam=1)
    
    observations = pm.Normal("d", mu=means[age_indexer], sd=sd, observed=data["d"])

# It's natural to consider our  data in terms of multiple factors...

In [ ]:
f, ax, = plt.subplots(figsize=(12, 6))
sns.violinplot(y="d", x="difficulty", hue="group", data=df, ax=ax, linewidth=4);

The `violinplot` shows the entire distribution,
which can be useful for noticing severe departures from normality,
but if we just want to compare the means,
a `pointplot` works better:

In [ ]:
f, ax, = plt.subplots(figsize=(12, 6))
sns.pointplot(y="d", x="difficulty", hue="group", data=df, ax=ax, scale=2, errwidth=6);

There is a hint of a different slope here,
suggesting there might be an interaction effect.

Both of these plots are effectively a "double `groupby`":

In [ ]:
df.groupby(["group", "difficulty"])["d"].mean()

# ... but then we need to update our inference procedure.

`scipy` does not provide functions for performing statistical tests
about multiple factors at once:
hence the `one` in `f_oneway`.

Later,
we'll see how this is done using a different Python library,
`statsmodels`.

The fact that the analytical statistical testing approach requires a new library
is another sign of its inflexibility.

We've already been working around this problem.
The `attention` dataset also has multiple possible grouping factors:
the `attention` column and the `solutions` column.
Previously, we either ignored one column
or looked at only rows where one of the two was fixed.

But we'll get a more complete understanding of our data
if we include all of the factors we measure.

# In pyMC, multi-way models are only a small adjustment: we define something like a list-of-lists for the parameters.

That is, we have one parameter for each combination of all factors.

$$
d \sim \text{Normal}(\mu[i, j], \sigma)
$$

In [ ]:
with pm.Model() as eeg_combined_model:
    means = pm.Normal("mus", mu=0, sd=1e6, shape=(3, 2))
    sigma = pm.Exponential("sigma", lam=0.1)
    
    observations = pm.Normal("d", mu=means[difficulty_indexer, age_indexer],
                             sd=sigma, observed=data["d"])

Technically, `means` is not a list-of-lists,
it's a specific type of list-of-lists called a `Tensor`
(even more accurately, a `TensorVariable`)
from the `theano` library.

It is also a `FreeRV`, just like a particular chihuahua is a `Dog` and also a `Mammal` and a `Pet`.

In [ ]:
import theano.tensor as tt

isinstance(means, tt.TensorVariable), isinstance(means, pm.model.FreeRV)

In [ ]:
with eeg_combined_model:
    eeg_combined_trace = pm.sample(draws=1000)
    eeg_combined_posterior = shared_util.samples_to_dataframe(eeg_combined_trace)

In [ ]:
print(eeg_combined_posterior.head())

In [ ]:
eeg_combined_posterior["mus"].iloc[0]

Each sample contains a mean for each combination of age group (column)
and task difficulty (row).

## As always, the first move is to visualize our posterior,

ideally in a manner similar to how we visualized our data.

In [ ]:
f, ax = plt.subplots(figsize=(12, 6))
[make_plot(eeg_combined_posterior.iloc[ii]["mus"], ax=ax, alpha=0.05)
 # posterior, plotted transparently
 for ii in random.sample(range(len(eeg_combined_posterior)), 200)];
ax.set_xticks([0, 1, 2]); ax.set_xlabel("Difficulty Index");
ax.set_ylabel("Group Average d")
sns.pointplot(x="difficulty", y="d", hue="age_group",  # real data
              data=data, ax=ax, axlabel=False, scale=2, errwidth=6); 

Notice how the slope of the line connecting difficulty 0 to difficulty 2 looks slightly steeper
for the yellow lines (the old age group)
than for the blue lines (the young age group)?

That suggests there is an interaction:
one way to phrase it is that the harder tasks are even harder for the older age group
than for the younger age group.

One thing that makes multi-way models harder is that the claims we are interested in
are not directly present in the group means.

That is, to get at the things we find interesting,
we typically need to `apply` some Python functions to the entries.

## We then compute the effects of factors from the entries of the `mu` array on each sample.

In [ ]:
def compute_delta_age_easy(mus):
    young_easy = mus[0, 0]
    old_easy = mus[0, 1]
    return old_easy - young_easy

def compute_delta_age_hard(mus):
    young_hard = mus[2, 0]
    old_hard = mus[2, 1]
    return old_hard - young_hard

In [ ]:
f, ax = plt.subplots(figsize=(12, 6))
sns.distplot(eeg_combined_posterior["mus"].apply(compute_delta_age_easy), label="Easy Task")
sns.distplot(eeg_combined_posterior["mus"].apply(compute_delta_age_hard), label="Hard Task",
             axlabel="Effect of Age on Performance"); ax.legend();

There appears to be an effect of age on performance only when the task is hard: an interaction!

In [ ]:
def compute_interaction_effect(mus):
    return compute_delta_age_hard(mus) - compute_delta_age_easy(mus)

In [ ]:
f, ax = plt.subplots(figsize=(12, 6))
sns.distplot(
    eeg_combined_posterior["mus"].apply(compute_interaction_effect),
    axlabel="Interaction Effect of Age and Difficulty");


## Other approaches that we might compare our results to are closer to MAP estimation.

_Maximum A Posteriori_ estimation means estimating the true values of the parameters by _maximizing_ the output of the _posterior_ as a function of the parameters, $\theta$:

$$
p\left(\theta \vert \text{data}\right)
$$

Treat the above as a function of $\theta$,
the values of all of the parameters,
and make it as large as possible.

Remember what `pyMC` uses to compute this value (up to a proportion that doesn't depend on the parameters:

$$
p\left(\theta \vert \text{data}\right) \propto p\left(\text{data} \vert \theta\right) p(\theta)
$$

that is, using the likelihood of the data given the parameters and the prior probability of the parameters.

In [ ]:
MAP_estimates = pm.find_MAP(model=eeg_combined_model)

In [ ]:
MAP_estimates

In [ ]:
f, ax = plt.subplots(figsize=(12, 6))
[make_plot(eeg_combined_posterior.iloc[ii]["mus"], ax=ax, alpha=0.05)
 for ii in random.sample(range(len(eeg_combined_posterior)), 200)];
ax.set_xticks([0, 1, 2]); ax.set_xlim([-0.5, 2.5]); ax.set_xlabel("Difficulty Index");
ax.set_ylabel("Group Average d")

ax.plot(MAP_estimates["mus"], lw=6)
ax.legend([make_line("C0"), make_line("C1")], ["Young MAP", "Old MAP"]);

In many, but not all, cases,
the parameters are close to the average values of the parameters
under the posterior,
as they are here.

MAP estimation is often done when "fully Bayesian" inference,
as we've been doing with `pyMC`,
is intractable.
Many machine learning algorithms
that are important in data science and psychology,
like neural networks trained by gradient descent
or any of the various flavors of linear or logistic regression,
can be framed as MAP estimation
in some (implicit) probabilistic model.

Any function that we can apply to individual posterior samples
can also be applied to our MAP estimates,
since a MAP estimate and a sample are both just possible values of the parameters of the model.

In [ ]:
compute_delta_age_easy(MAP_estimates["mus"])

In [ ]:
compute_delta_age_hard(MAP_estimates["mus"])

In [ ]:
compute_interaction_effect(MAP_estimates["mus"])

The result is often the most likely value of the function being computed,
under the posterior, but not always:
if the function being used maps multiple inputs to the same output,
then there can be a difference.

In that case, checking whether the MAP estimate also maximizes the probability
of a given statistic is much harder,
but for the case of these factor effects, it does.

## We could alternatively have written our model directly in terms of the effects.

$$
\text{baseline} = \mu[0, 0] \\
\Delta_\text{old age} = \mu[0, 1] - \text{baseline} \\
\Delta_\text{hard difficulty} = \mu[2, 0] - \text{baseline} \\
\Delta_\text{interact old and hard} = \mu[2, 1] - (\text{baseline} + \Delta_\text{old age} + \Delta_\text{hard difficulty})\\
$$

$$
\text{d} \sim \text{Normal}(\text{baseline}
+ \text{is_young}\cdot\text{is_easy}\cdot\Delta_\text{old easy}
+ \text{is_young}\cdot\text{is_hard}\cdot\Delta_\text{young hard}
+ \text{is_young}\cdot\text{is_hard}\cdot\Delta_\text{old hard},
\sigma)
$$

$\text{is_old}$ is a binary variable that records whether the participant is in the older age group. $\text{is_hard}$ is a binary variable that records whether the task was in the hard difficulty group.

$\text{is_old_and_hard}$ is a binary variable that records whether both of the above are true (aka `and`).

See below.

In [ ]:
eeg_interaction_subset = data[data["difficulty"] != 1]

is_old = eeg_interaction_subset["age_group"] == 1
is_hard = eeg_interaction_subset["difficulty"] == 2
is_old_hard = is_old & is_hard

In [ ]:
with pm.Model() as linear_model:
    sigma = pm.Exponential("sigma", lam=1)
    
    baseline = pm.Normal("baseline", mu=0, sd=1e1)
    
    delta_old_easy = pm.Normal("delta_old_easy", mu=0, sd=1e1)
    delta_hard_young = pm.Normal("delta_hard_young", mu=0, sd=1e1)
    
    interact_old_hard = pm.Normal("interact_old_hard", mu=0, sd=1e1)
    
    d = pm.Normal("d",
                  mu=baseline
                  + delta_old_easy * is_old
                  + delta_hard_young * is_hard
                  + interact_old_hard * is_old_hard,
                  sd=sigma,
                  observed=eeg_interaction_subset["d"])

In [ ]:
linear_model_trace = shared_util.sample_from(linear_model)

In [ ]:
pm.plot_posterior(linear_model_trace, figsize=(12, 12), text_size=16,
                  varnames=["baseline", "delta_old_easy",
                            "delta_hard_young", "interact_old_hard"],
                  ref_val=[4.5, 0, 0, 0]);

The advantage of this approach is that the random variables we define are more closely related to the inferences we are trying to draw.

The disadvantage is that these models get very complicated very quickly as we add more factors.
They are tedious to specify by hand, especially inside a `pyMC` model.
It's usually easier to define the quantities we want to infer afterwards,
as Python functions we apply to samples.

This style of writing a model is called a _linear model_,
because the means, as a function of the data, look like
$$y=m\cdot x +b $$

For example, for the datapoints from the young participants doing the hard task,
we have that the mean performance is

$$
\hat{d} = \Delta_\text{hard} \cdot \text{is_hard} + \text{baseline}
$$

where $\hat{d}$, the mean, is our "dependent variable" $y$,
$\text{is_hard}$ is the "independent variable" $x$,
$\Delta_\text{hard}$ is the "slope" $m$,
and $\text{baseline}$ is the "intercept" $b$.

So the other advantage to this way of specifying models is that
we can more easily relate these kinds of models to a broad array of models called
_generalized linear models_ or _GLM_s.

Next week, we'll look at a different kind of basic linear model,
a _linear regression model_,
and then see how the GLM framework includes both,
along with lots of other kinds of models.